## Essa parte é só para funcionar no colab

In [ ]:
! git clone https://github.com/suny-downstate-medical-center/S1_netpyne.git

Cloning into 'S1_netpyne'...
remote: Enumerating objects: 29875, done.
remote: Counting objects: 100% (21/21), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 29875 (delta 0), reused 16 (delta 0), pack-reused 29854
Receiving objects: 100% (29875/29875), 2.35 GiB | 17.50 MiB/s, done.
Resolving deltas: 100% (6721/6721), done.
Checking out files: 100% (25878/25878), done.


In [ ]:
! pip install neuron

     |████████████████████████████████| 12.6 MB 219 kB/s 


In [ ]:
! pip install netpyne

     |████████████████████████████████| 312 kB 4.3 MB/s 


In [ ]:
cd S1_netpyne/info/

/content/S1_netpyne/info


## Bibliotecas

In [ ]:
import json
import os, sys
import pandas as pd
import numpy as np

# Cell Number per population from BBP S1 model

In [ ]:
Epops= ['L23_PC', 'L4_PC', 'L4_SS', 'L4_SP', 
             'L5_TTPC1', 'L5_TTPC2', 'L5_STPC', 'L5_UTPC',
             'L6_TPC_L1', 'L6_TPC_L4', 'L6_BPC', 'L6_IPC', 'L6_UTPC']

with open('anatomy/S1-cells-distributions-Rat.txt') as mtype_file:
    mtype_content = mtype_file.read()       

popParamLabels = []
Ipops = []

for line in mtype_content.split('\n')[:-1]:
    metype, mtype, etype, n, m = line.split()
    if mtype not in popParamLabels:
        popParamLabels.append(mtype)
        if mtype not in Epops:
            Ipops.append(mtype)
n2 = 0
metag = {}
popNumber2 = {}
cellNumber = {} 
popLabel = {} 
popLabelEl = {} 
meParamLabels = {} 
for line in mtype_content.split('\n')[:-1]:
    metype, mtype, etype, n, m = line.split()
    cellNumber[metype] = int(n)
    popLabel[metype] = mtype
    popLabelEl[metype] = etype
    popNumber2[mtype] = int(m)
    metag[metype] = n2    
    n2 = n2 + 1

with open('anatomy/mtype_map.tsv') as mtype_map_file:
    mtype_map_content = mtype_map_file.read()    
mtype_map = {}
mtype_map2 = {}
for line in mtype_map_content.split('\n')[:-1]:
    n, mtype = line.split()
    mtype_map[mtype] = int(n)
    mtype_map2[int(n)] = mtype    

# Anatomy pathways from Netconnections_mean.json (Cell2015)

In [ ]:
# initialize prob and weight matrices
# format: pmat[presynaptic_pop][postsynaptic_pop] 
data = {}
data['BBP_S1'] = {}
with open('anatomy/Netconnections_mean.json', 'r') as f:
    data['BBP_S1']['connProb'] = json.load(f) 
connDataSource = {}
connDataSource['E->E/I'] = 'BBP_S1' 
connDataSource['I->E/I'] = 'BBP_S1' 
pmat = {}  # probability of connection matrix full mc2
connNumber = {}        # ~ "total_conn_count"
lmat = {}  # length constant (lambda) for exp decaying prob conn (um) matrix
a0mat = {} # probability of connection matrix dist 2D  = 0 um
d0 = {} #  matrix min to fit exp dist 2D [12.5,25,...,150]
dfinal = {} #  matrix max to fit exp dist 2D [125,...,375]
pmat12um = {} # probability of connection matrix dist 2D -> 12.5um +- 12.5um
pmat25um = {} # probability of connection matrix dist 2D -> 25um +- 25um
pmat50um = {}
pmat75um = {}
pmat100um = {}
pmat125um = {}
pmat150um = {}
pmat175um = {}
pmat200um = {}
pmat225um = {}
pmat250um = {}
pmat275um = {}
pmat300um = {}
pmat325um = {}
pmat350um = {}
pmat375um = {} 
for p in Epops + Ipops:
    pmat[p] = {}
    connNumber[p] = {}  
    lmat[p] = {}
    a0mat[p] = {}  
    d0[p] = {}
    dfinal[p] = {}
    pmat12um[p] = {}
    pmat25um[p] = {}
    pmat50um[p] = {}
    pmat75um[p] = {}
    pmat100um[p] = {}
    pmat125um[p] = {}
    pmat150um[p] = {}
    pmat175um[p] = {}
    pmat200um[p] = {}
    pmat225um[p] = {}
    pmat250um[p] = {}
    pmat275um[p] = {}
    pmat300um[p] = {}
    pmat325um[p] = {}
    pmat350um[p] = {}
    pmat375um[p] = {}     

In [ ]:
synperconnNumber = {}        # ~ "mean_number_of_synapse_per_connection"
synperconnNumberStd = {}   
for p in Epops + Ipops:   
    synperconnNumber[p] = {}
    synperconnNumberStd[p] = {}

for pre in Epops+Ipops:
    for post in Epops+Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:
            synperconnNumber[pre][post] = 0
            synperconnNumberStd[pre][post] = 0

In [ ]:
# LOAD DATA FROM .TSV files resume
with open('anatomy/synNumberperconex.dat') as synNumber_file:
    synNumber_content = synNumber_file.read()
    
for line in synNumber_content.split('\n')[:-1]:
    n, m, mean, stdev, synNumber, proj = line.split()
    pre = mtype_map2[int(n)]
    post = mtype_map2[int(m)]   
    proj = '%s:%s' % (pre, post)
    if proj in data['BBP_S1']['connProb']:    
        synperconnNumber[pre][post] = np.ceil(float(mean))
        synperconnNumberStd[pre][post] = float(stdev)

In [ ]:
# LOAD WITH PANDAS DATA FROM json files 
anatomy_data = json.loads(open("anatomy/pathways_anatomy_factsheets_simplified.json").read())
columnNames2 = ["total_synapse_count", "mean_number_of_synapse_per_connection", "number_of_convergent_neuron_mean"]
columnNames = ["From Cell", "To Cell"] + list(anatomy_data[list(anatomy_data.keys() )[0]].keys())
df = pd.DataFrame(columns=columnNames, data = [[k.split(":")[0], k.split(":")[1]] + list(v.values()) for k, v in anatomy_data.items()    ])
df = df.sort_values(by=['From Cell'])
df = df.drop(columns=['number_of_convergent_neuron_std', 'connection_probability', 'number_of_divergent_neuron_std', 'total_synapse_count',
 'common_neighbor_bias', 'number_of_convergent_neuron_mean', 'number_of_divergent_neuron_mean'])
for conn in range(1932):
    pre = df["From Cell"][conn]
    post = df["To Cell"][conn]
    proj = '%s:%s' % (pre, post)
    if proj in data['BBP_S1']['connProb']:
        synperconnNumber[pre][post] = np.ceil(df["mean_number_of_synapse_per_connection"][conn])
        synperconnNumberStd[pre][post] = df["number_of_synapse_per_connection_std"][conn]
df.head()

,From Cell,To Cell,mean_number_of_synapse_per_connection,number_of_synapse_per_connection_std
1616,L1_DAC,L1_SLAC,12.0,3.9
839,L1_DAC,L1_HAC,15.0,6.3
558,L1_DAC,L23_NGC,7.0,1.9
1768,L1_DAC,L4_SP,5.7,1.7
1769,L1_DAC,L4_SS,6.1,2.7


In [ ]:
# For missing data use mean of Layer-Type:Layer-Type projections
layersT = ['L1e', 'L2e', 'L4e', 'L5e', 'L6e', 'L1i', 'L2i', 'L4i', 'L5i', 'L6i']
synperconnNumberT = {}
synperconnNumberN = {}
for pre in layersT:
    synperconnNumberT[pre] = {}
    synperconnNumberN[pre] = {}
    for post in layersT:
        synperconnNumberT[pre][post] = 0
        synperconnNumberN[pre][post] = 0           
    
for pre in Epops+Ipops:
    for post in Epops+Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:

            if pre in Epops:
                pre2 = str(pre[0:2]) + 'e'
            else:
                pre2 =  str(pre[0:2]) + 'i'
            if post in Epops:
                post2 =  str(post[0:2]) + 'e'
            else:
                post2 =  str(post[0:2]) + 'i'

            if synperconnNumber[pre][post] > 0:
                synperconnNumberT[pre2][post2] = synperconnNumberT[pre2][post2] + synperconnNumber[pre][post]
                synperconnNumberN[pre2][post2] = synperconnNumberN[pre2][post2] + 1

for pre in Epops+Ipops:
    for post in Epops+Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:
            if synperconnNumber[pre][post] == 0:
                if pre in Epops:
                    pre2 = str(pre[0:2]) + 'e'
                else:
                    pre2 =  str(pre[0:2]) + 'i'

                if post in Epops:
                    post2 =  str(post[0:2]) + 'e'
                else:
                    post2 =  str(post[0:2]) + 'i'

                synperconnNumber[pre][post] = np.ceil(synperconnNumberT[pre2][post2]/synperconnNumberN[pre2][post2])  #mean of Layer-Type:Layer-Type projections

In [ ]:
number = 0 
for pre in Epops:
    for post in Epops+Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:            
            number = number + synperconnNumber[pre][post] * float(data['BBP_S1']['connProb'][proj]['conn_total'])
print('syn total Exc = %.0f' % number)

syn total Exc = 27614163


In [ ]:
numberI = 0 
for pre in Ipops:
    for post in Epops+Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:            
            numberI = numberI + synperconnNumber[pre][post] * float(data['BBP_S1']['connProb'][proj]['conn_total'])
print('syn total Inh = %.0f' % numberI)

syn total Inh = 10423033


In [ ]:
for conn in range(1932):
    pre = df["From Cell"][conn]
    post = df["To Cell"][conn]
    proj = '%s:%s' % (pre, post)
    if proj in data['BBP_S1']['connProb']:
        if pre in Ipops:
            synperconnNumber[pre][post] = synperconnNumber[pre][post]-1

# number = 0 
# for conn in range(1932):
#     pre = df["From Cell"][conn]
#     post = df["To Cell"][conn]
#     proj = '%s:%s' % (pre, post)
#     if proj in data['BBP_S1']['connProb']:
#         error = 100*(synperconnNumber[pre][post]-df["mean_number_of_synapse_per_connection"][conn])/df["mean_number_of_synapse_per_connection"][conn]
#         if pre in Ipops:
#             print ('%s:%s - Json = %.2f - tsv = %.2f - error = %.1f percent (%s)' % (pre, post,df["mean_number_of_synapse_per_connection"][conn],synperconnNumber[pre][post],error,data['BBP_S1']['connProb'][proj]['conn_total']))
#             number = number + 1
numberI = 0 
for pre in Ipops:
    for post in Epops+Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:            
            numberI = numberI + synperconnNumber[pre][post] * float(data['BBP_S1']['connProb'][proj]['conn_total'])
print('syn total Inh = %.0f' % numberI)

syn total Inh = 9684705


In [ ]:
number = 0 
for pre in Epops+Ipops:
    for post in Epops+Ipops: 
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:           
            if synperconnNumber[pre][post] > 0:
                number = number + 1
number

1940

# Physiology pathways from Cell2015 (table S6 and portal data)

In [ ]:
# LOAD WITH PANDAS DATA FROM json files 
physiology_data = json.loads(open("physiology/pathways_physiology_factsheets_simplified.json").read())
physColumnNames = ["Connection", "From Cell", "From Layer", "From Type", "To Cell", "To Layer", "To Type"] + list(physiology_data[list(physiology_data.keys() )[0]].keys())
df2 = pd.DataFrame(columns=physColumnNames, data = [[k,  k.split(":")[0],  k.split(":")[0].split("_")[0], k.split(":")[0].split("_")[1], k.split(":")[1], k.split(":")[1].split("_")[0], k.split(":")[1].split("_")[1]] + list(v.values()) for k, v in physiology_data.items()    ])
df2 = df2.sort_values(by=['Connection'])
df2.head()

,Connection,From Cell,From Layer,From Type,To Cell,To Layer,To Type,gsyn_mean,epsp_mean,risetime_std,f_std,gsyn_std,u_std,decay_mean,latency_mean,failures_mean,u_mean,d_std,synapse_type,space_clamp_correction_factor,latency_std,decay_std,cv_psp_amplitude_std,risetime_mean,cv_psp_amplitude_mean,epsp_std,d_mean,f_mean,failures_std
1558,L1_DAC:L1_DAC,L1_DAC,L1,DAC,L1_DAC,L1,DAC,0.36,2.1,0.85,5.0,0.15,0.072,38.0,1.70,6.3,0.26,220.0,"Inhibitory, depressing",Synaptic conductance not measured experimentally,0.6500,11.0,0.120,5.5,0.49,0.45,750,22.0,7.8
1841,L1_DAC:L1_DLAC,L1_DAC,L1,DAC,L1_DLAC,L1,DLAC,0.41,2.2,0.00,3.8,0.12,0.053,36.0,1.10,0.0,0.29,170.0,"Inhibitory, depressing",Synaptic conductance not measured experimentally,0.0000,0.0,0.000,6.1,0.39,0.00,820,24.0,0.0
1405,L1_DAC:L1_HAC,L1_DAC,L1,DAC,L1_HAC,L1,HAC,0.37,2.4,0.90,4.7,0.15,0.068,29.0,1.60,4.3,0.27,210.0,"Inhibitory, depressing",Synaptic conductance not measured experimentally,0.4700,2.5,0.100,5.8,0.46,0.94,770,22.0,4.6
662,L1_DAC:L1_NGC-DA,L1_DAC,L1,DAC,L1_NGC-DA,L1,NGC-DA,0.26,2.6,0.29,4.0,0.12,0.055,21.0,0.76,13.0,0.22,170.0,"Inhibitory, depressing",Synaptic conductance not measured experimentally,0.0033,1.7,0.026,3.8,0.62,0.52,610,19.0,3.3
513,L1_DAC:L1_NGC-SA,L1_DAC,L1,DAC,L1_NGC-SA,L1,NGC-SA,0.33,2.6,0.58,4.1,0.13,0.059,26.0,1.50,11.0,0.25,180.0,"Inhibitory, depressing",Synaptic conductance not measured experimentally,0.5900,3.3,0.036,4.4,0.58,0.75,710,21.0,1.6


In [ ]:
with open('physiology/physiology_synTypes.dat') as physiology_file:
    physiology_content = physiology_file.read()

parameters_name = {}   
ConnTypeNumber = -1  
parameters_syn = {}
syntypes = {}
for line in physiology_content.split('\n')[:-1]:    
    if ConnTypeNumber < 0:
        n2 = 0
        parameters_list = line.split()   
        for parameter in parameters_list:
            parameters_name[n2] = str(parameter)
            n2 = n2 + 1   
    else:
        n2 = 0
        parameters_list = line.split()   
        for parameter in parameters_list:
            if n2 == 0:
                parameters_syn[parameters_name[n2],ConnTypeNumber] = int(parameter)
                syntypes[ConnTypeNumber] = int(parameter)
            elif n2 > 10:
                parameters_syn[parameters_name[n2],ConnTypeNumber] = str(parameter)                
            else:
                parameters_syn[parameters_name[n2],ConnTypeNumber] = float(parameter)
            n2 = n2 + 1
                
    ConnTypeNumber = ConnTypeNumber + 1
    
parameters_name[0]

'ConnType'

In [ ]:
physColumnNames = [parameters_name[x] for x in parameters_name]
dfS6 = pd.DataFrame(index=syntypes, columns=physColumnNames)
for syntype in syntypes:
    for name in physColumnNames:    
        dfS6.loc[syntype][name] = parameters_syn[name,syntype]
dfS6

,ConnType,gsyn,gsynStd,decay,decayStd,use,useStd,dep,depStd,fac,facStd,s-types,type
0,0,0.83,0.2,10.4,6.1,0.16,0.1,45,21,376,253,I1,IE
1,1,0.83,0.55,8.3,2.2,0.25,0.13,706,405,21,9,I2,II
2,3,0.91,0.61,10.4,6.1,0.16,0.1,45,21,376,253,I1,IE
3,4,0.91,0.61,8.3,2.2,0.25,0.13,706,405,21,9,I2,IE
4,5,0.91,0.61,6.44,1.7,0.32,0.14,144,80,62,31,I3,IE
5,8,0.75,0.32,8.3,2.2,0.25,0.13,706,405,21,9,I2,IE
6,9,0.75,0.32,8.3,2.2,0.3,0.08,1250,520,2,4,I2,IE
7,10,0.91,0.61,8.3,2.2,0.14,0.05,875,285,22,5,I2,IE
8,11,0.83,0.2,10.4,6.1,0.25,0.13,706,405,21,9,I3,IE
9,12,0.91,0.61,8.3,2.2,0.25,0.13,706,405,21,9,I2,IE


In [ ]:
with open('physiology/matrixsyntypes.dat') as synNumber_file:
    synNumber_content = synNumber_file.read()

ConnTypes = {}
ConnTypesNumber = {}      
for p in Epops + Ipops:   
    ConnTypesNumber[p] = {}
    ConnTypes[p] = {}

for pre in Epops+Ipops:
    for post in Epops+Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:
            ConnTypesNumber[pre][post] = 0
            ConnTypes[pre][post] = 0
    
for line in synNumber_content.split('\n')[:-1]:
    n, m, number, number2, number3, number4 = line.split()
    pre = mtype_map2[int(n)]
    post = mtype_map2[int(m)]
    proj = '%s:%s' % (pre, post)
    if proj in data['BBP_S1']['connProb']:
        if int(number3)!=50:
            ConnTypesNumber[pre][post] = 3
            ConnTypes[pre][post] = [int(number), int(number2), int(number3)]
        elif int(number2)!=50:
            ConnTypesNumber[pre][post] = 2
            ConnTypes[pre][post] = [int(number), int(number2)]
        elif int(number)!=50:
            ConnTypesNumber[pre][post] = 1
            ConnTypes[pre][post] = [int(number)]

In [ ]:
# ConnTypes II
for pre in Ipops:
    for post in Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:                
            if ConnTypesNumber[pre][post] == 0:
#                 print ('ConnNumberTypes[%s][%s] = %d' % (pre, post, ConnTypesNumber[pre][post])) 
                ConnTypesNumber[pre][post] = 1
                ConnTypes[pre][post] = [1]     

In [ ]:
# ConnTypes IE
for pre in Ipops: # L1 
    for post in Epops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:           
            if str(pre[0:2]) == 'L1':
                ConnTypesNumber[pre][post] = 1
                ConnTypes[pre][post] = [1]    
                
for pre in Ipops:
    for post in Epops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:   
            if str(pre[-3:]) == 'NGC':
                ConnTypesNumber[pre][post] = 1
                ConnTypes[pre][post] = [11]    
                
ConnTypesNumber['L23_MC']['L6_IPC'] = 1
ConnTypes['L23_MC']['L6_IPC'] = [9]

ConnTypesNumber['L6_DBC']['L6_SBC'] = 1
ConnTypes['L6_DBC']['L6_SBC'] = [0]

for pre in Ipops:
    for post in Epops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:   
            if str(pre[-3:]) == 'ChC':
                ConnTypesNumber[pre][post] = 1
                ConnTypes[pre][post] = [12]  

for pre in Ipops:
    for post in Epops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:   
            if str(pre[-3:]) == 'DBC' or str(pre[-3:]) == '_BP' or str(pre[-3:]) == 'BTC':
                ConnTypesNumber[pre][post] = 1
                ConnTypes[pre][post] = [8]  
for pre in Ipops:
    for post in Epops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:   
            if str(pre[-3:]) == 'SBC':
                ConnTypesNumber[pre][post] = 3
                ConnTypes[pre][post] = [4, 5, 3]   
                
for pre in Ipops:
    for post in Epops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:   
            if str(pre[-3:]) == 'LBC' or str(pre[-3:]) == 'NBC':
                ConnTypesNumber[pre][post] = 2
                ConnTypes[pre][post] = [5, 10]

In [ ]:
# ConnTypes EI
ConnTypesNumber['L4_SS']['L1_HAC'] = 1
ConnTypes['L4_SS']['L1_HAC'] = [117]
ConnTypesNumber['L5_TTPC1']['L1_HAC'] = 1
ConnTypes['L5_TTPC1']['L1_HAC'] = [117]
ConnTypesNumber['L5_TTPC2']['L1_HAC'] = 1
ConnTypes['L5_TTPC2']['L1_HAC'] = [117]
ConnTypesNumber['L5_UTPC']['L1_DAC'] = 1
ConnTypes['L5_UTPC']['L1_DAC'] = [117]
ConnTypesNumber['L6_TPC_L1']['L23_NGC'] = 1
ConnTypes['L6_TPC_L1']['L23_NGC'] = [117]

In [ ]:
for pre in Epops:
    for post in Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:   
            if ConnTypesNumber[pre][post] == 2:
                if 117 in ConnTypes[pre][post]:
                    ConnTypes[pre][post] = [117, 114]
                elif 134 in ConnTypes[pre][post]:
                    ConnTypes[pre][post] = [134, 115]          

In [ ]:
number = 0
for pre in Epops+Ipops:
    for post in Epops+Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:   
            if ConnTypesNumber[pre][post] > 0:
              #print ('ConnNumberTypes[%s][%s] = %d' % (pre, post, ConnTypesNumber[pre][post])) 
              #print (ConnTypes[pre][post])
              number = number + 1
#number      

## CA1 MOD

In [ ]:
synmechparams = {}
for pre in Epops+Ipops:
    for post in Epops+Ipops:
        proj = '%s:%s' % (pre, post)
        if proj in data['BBP_S1']['connProb']:  
          if pre in Epops:
            mod = 'DetAMPANMDA'
            tau = 'tau_d_AMPA'
          else:
            mod = 'DetGABAAB'
            tau = 'tau_d_GABAA'

          conn = connType[ConnTypes[pre][post][0]]
          
          synmechparams[proj] = {'mod': mod,
                                 tau: dfS6['decay'][conn],
                                 #'tau_d_NMDA': 61.0, 
                                 'Use': dfS6['use'][conn],
                                 'Dep': dfS6['dep'][conn], 
                                 'Fac': dfS6['fac'][conn], 
                                 'gmax': dfS6['gsyn'][conn]}

In [ ]:
synmechparams

{'L23_PC:L23_PC': {'Dep': 671.0,
  'Fac': 17.0,
  'Use': 0.46,
  'gmax': 0.68,
  'mod': 'DetAMPANMDA',
  'tau_d_AMPA': 1.74},
 'L23_PC:L4_PC': {'Dep': 671.0,
  'Fac': 17.0,
  'Use': 0.5,
  'gmax': 0.72,
  'mod': 'DetAMPANMDA',
  'tau_d_AMPA': 1.74},
 'L23_PC:L4_SS': {'Dep': 671.0,
  'Fac': 17.0,
  'Use': 0.5,
  'gmax': 0.72,
  'mod': 'DetAMPANMDA',
  'tau_d_AMPA': 1.74},
 'L23_PC:L4_SP': {'Dep': 671.0,
  'Fac': 17.0,
  'Use': 0.5,
  'gmax': 0.72,
  'mod': 'DetAMPANMDA',
  'tau_d_AMPA': 1.74},
 'L23_PC:L5_TTPC1': {'Dep': 671.0,
  'Fac': 17.0,
  'Use': 0.5,
  'gmax': 0.72,
  'mod': 'DetAMPANMDA',
  'tau_d_AMPA': 1.74},
 'L23_PC:L5_TTPC2': {'Dep': 671.0,
  'Fac': 17.0,
  'Use': 0.5,
  'gmax': 0.72,
  'mod': 'DetAMPANMDA',
  'tau_d_AMPA': 1.74},
 'L23_PC:L5_STPC': {'Dep': 671.0,
  'Fac': 17.0,
  'Use': 0.5,
  'gmax': 0.72,
  'mod': 'DetAMPANMDA',
  'tau_d_AMPA': 1.74},
 'L23_PC:L5_UTPC': {'Dep': 671.0,
  'Fac': 17.0,
  'Use': 0.5,
  'gmax': 0.72,
  'mod': 'DetAMPANMDA',
  'tau_d_AMPA': 1.7

In [ ]:
#for pre in Epops+Ipops:
#    for post in Epops+Ipops:
#        proj = '%s:%s' % (pre, post)
#        if proj in data['BBP_S1']['connProb']:  
#          if pre in Epops:
#            mod = 'DetAMPANMDA'
#          else:
#            mod = 'DetGABAAB'
#
#          conn = connType[ConnTypes[pre][post][0]]
#          
#          netParams.synMechParams[proj] = {'mod': mod,
#                                           'tau_d_AMPA': dfS6['decay'][conn],
#                                           #'tau_d_NMDA': 61.0, 
#                                           'Use': dfS6['use'][conn],
#                                           'Dep': dfS6['dep'][conn], 
#                                           'Fac': dfS6['fac'][conn], 
#                                           'gmax': dfS6['gsyn'][conn]}